# Building a Facial Recognition system from pre-trained networks

You have been given extensive examples in the lab webpage, so please refer to those while completing this notebook.  Begin by installing the git repo.

### Understanding your networks

Take a picture with your phone of somebody in the class, and upload that picture to this notebook.  Open the image, and display it with `plt.imshow(img)`.

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
from matplotlib import pyplot as plt
import torch
import numpy as np
import s3fs
from sklearn.neighbors import NearestNeighbors

ModuleNotFoundError: No module named 's3fs'

In [ ]:
img = Image.open('origProbe.jpg')
plt.imshow(img)

Now, run it through your facial detector.  The output will be a tensor, suppose it's called `img_cropped`. Print out its shape, and then display the image. You can display it with `plt.imshow(img_cropped.permute(1,2,0))`.

In [ ]:
mtcnn = MTCNN(keep_all=True)
mtcnn.eval()
cropped_img = mtcnn(img)
print(f'shape is {cropped_img.shape}')

fig, axs = plt.subplots(1, 5, figsize=(18, 3))

for i in range(5):
    axs[i].imshow(cropped_img[i].permute(1,2,0))
    axs[i].axis('off')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

Now, create an embedding of that face, and print out its shape.

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cuda')
embeds = resnet(cropped_img.to('cuda'))
embeds = embeds.detach().cpu().numpy()
print(f'Shape is {embeds.shape}')

### Getting your Gallery

Using the S3 bucket `s3://mlspace-data-521454461163/project/10FacialRecognition/datasets/midsfaces/`, for all midshipmen currently taking SD312, create a list of all the images as numpy arrays, and then a numpy array of all the embeddings of their cropped faces, where `embeddings[0]` is a row vector containing the embedding of `images[0]` from your list of images.

In [ ]:
sd312Alphas = ['M250024','M250966','M251344','M252310','M252670','M252916','M253078','M253762','M253828','M255514','M256168','M256258','M256264','M256294','M256774','M257062','M250522','M252016','M252754','M253306','M253384','M254230','M254530','M254572','M256978','M250072','M250420','M250996','M251632','M252130','M253036','M253120','M254542','M254932','M255556','M255748','M255994','M256354','M256894']
fs = s3fs.S3FileSystem()
all_filenames = fs.ls('s3://mlspace-data-521454461163/project/10FacialRecognition/datasets/midsfaces/')
gallery = []
for filename in all_filenames:
    alpha = filename.split('/')[-1].split('.')[0]
    if alpha in sd312Alphas:
        with fs.open('s3://'+filename) as s3_object:
            gallery.append(np.array(Image.open(s3_object)))

In [ ]:
gallery_embeddings = np.zeros((len(gallery), 512))
mtcnn = MTCNN()
for i in range(len(gallery)):
    cropped = mtcnn(gallery[i])
    gallery_embeddings[i,:] = resnet(cropped.unsqueeze(0).to('cuda')).detach().cpu().numpy()

`.fit()` a sklearn `NearestNeighbor` object to your trained embeddings.  Let's keep the top-5 closest faces.

In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(gallery_embeddings)

### Building your recognition system

Now, given your picture of someone in your class, find the five faces from SD312 which are closest in embedding space to your picture.  Write a function that displays first the probe image, and then the five gallery images closest to that probe image.

In [ ]:
results = nn.kneighbors(embeds)
print(results)

In [ ]:
def showMatches(probe, galleries):
    # Create a figure with 1 row and 6 columns for subplots
    fig, axs = plt.subplots(1, 6, figsize=(18, 3))

    # Plot each image on its corresponding subplot
    axs[0].imshow(probe)
    axs[0].axis('off')  # Turn off axis for cleaner display
    axs[0].set_title('Probe')

    # Draw a vertical line after the first image
    axs[0].axvline(x=0.5, color='black', linestyle='--')

    for i in range(len(galleries)):
        axs[i+1].imshow(galleries[i])
        axs[i+1].axis('off')

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
bestGallery=[]
for i in range(5):
    bestGallery.append([gallery[i] for i in results[1][i]])
    showMatches(cropped_img[0].permute(1,2,0), bestGallery[0])

Collect more pictures of people in the class.  Find some with people wearing a hat, or a Halloween costume, or a covid mask, or a terrible mustache.  How well do they work?

Expand to the full class of 2025 for your gallery images.  Then expand to all midshipmen in the dataset.  How does your accuracy change?